<a href="https://colab.research.google.com/github/Joey-mi/CPSC572---Electron-Packages/blob/main/HadoopMapReduce_Co_Occurence_Counter_for_Dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create the input and output files on hadoop DFS then put our test data in
!hadoop fs -mkdir -p /outputs
!hadoop fs -mkdir -p /inputs
!hdfs dfs -rm -r /inputs/nonDevDependencies
!hadoop fs -put nonDevDependencies /inputs

Deleted /inputs/nonDevDependencies
2024-03-26 06:22:18,277 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2024-03-26 06:22:18,544 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [ ]:
%%writefile mapper3.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys
import os

for line in sys.stdin: # reads from stdin

    # Convert string to list
    array = list(map(str.strip, line.strip('][').replace('"', '').split(',')))

    # Clean up strings in list
    array = list(map(lambda x: x.replace("'", "").replace('"', '').replace("@", "").replace('[', '').replace(']', ''), array))

    n = len(array)

    # Since arrays are already sorted, we iterate through them in O(n^2) to get all possible pairs
    for i in range(n):
        for j in range(i+1, n):
            print(array[i][0] + "," + array[i] + "," + array[j])


Overwriting mapper3.py


In [ ]:
%%writefile reducer3.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys
import os

pair = None
pairedCount = 0

for line in sys.stdin:

    intKv = line.strip().split(",") # item1[0],item1,item2
    # Pass if we get an empty string
    if len(intKv) != 3:
        continue

    parsedLine = intKv[1].strip()+"@"+intKv[2].strip()
    kv = [parsedLine, "1"] # item1,item2<\t>count


    # When it's a different item pair, we will store the entry
    if kv[0] != pair:
        if pair is not None:
            if pairedCount > 1:
                print(f"{pair}, {pairedCount}")

        # Reset the variables
        pair = kv[0]
        pairedCount = 1
    # If it's the same item pair, then we increment the count
    else:
        pairedCount += 1

# Output the last pair at the end
if pair is not None:
    if pairedCount > 1:
        print(f"{pair}, {pairedCount}")

Overwriting reducer3.py


In [ ]:
# Remove the output file if already present
!hdfs dfs -rm -r /outputs/project572

# For the KeyFieldBasedPartitioner, specify we have three fields separated by "," and use the first field to partition with option k1,1
# Specify 26 reduce tasks for each letter of the alphabet
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D stream.map.output.field.separator=, \
    -D stream.num.map.output.key.fields=3 \
    -D map.output.key.field.separator=, \
    -D mapred.text.key.partitioner.options=-k1,1 \
    -D mapred.reduce.tasks=26 \
    -file $PWD/mapper3.py\
    -file $PWD/reducer3.py\
    -input /inputs/nonDevDependencies/noDevCoOccurenceLists.txt \
    -output /outputs/project572 \
    -mapper mapper3.py \
    -reducer reducer3.py \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner



Deleted /outputs/project572
2024-03-26 06:37:20,490 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/training/Assignment2/mapper3.py, /training/Assignment2/reducer3.py, /tmp/hadoop-unjar1752999774726923773/] [] /tmp/streamjob3308468320850206538.jar tmpDir=null
2024-03-26 06:37:23,191 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.3:8032
2024-03-26 06:37:23,619 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.5:10200
2024-03-26 06:37:23,701 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.3:8032
2024-03-26 06:37:23,702 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.5:10200
2024-03-26 06:37:24,172 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1711418519987_0005
2024-03-26 06:37:24,449 INFO sasl.SaslDataTransferClient

In [ ]:
# Get the output from HDFS to local store
!hadoop fs -copyToLocal /outputs/project572 countsFolder


2024-03-26 06:41:02,490 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
